<a href="https://colab.research.google.com/github/nallagondu/postgres/blob/main/postgres_training_5_Monitoring_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PostgreSQL Monitoring Guide: Comprehensive Overview**

---

### 🔍 Overview

PostgreSQL monitoring ensures that the database runs smoothly, identifies performance bottlenecks, and maintains availability. In this session, three levels of monitoring were discussed:

1. **Grafana + Prometheus Dashboard (Proactive/Reactive Monitoring)**
2. **Manual Monitoring via CLI Scripts & Commands**
3. **PGAdmin GUI (To be covered in next session)**

---

### 📊 Grafana + Prometheus Architecture

**Components**:

* **Node Exporter**: Agent on DB servers collecting CPU, memory, disk, and process data
* **Prometheus Server**: Pulls metrics from Node Exporter and stores time-series data
* **Grafana**: Dashboards that visualize Prometheus data

**Metrics Collected**:

| Metric Category      | Examples                            |
| -------------------- | ----------------------------------- |
| CPU Usage            | %CPU, Load Average, Per Core Stats  |
| Memory Usage         | Total, Used, Free, Cache            |
| Disk Usage           | File system size, Used space        |
| PostgreSQL Health    | DB Uptime, Connection Count, Role   |
| Replication          | Lag, Status, Role (Primary/Standby) |
| Blocking & Deadlocks | Blocking Queries, Deadlock Events   |
| Table Bloat          | %Bloat, Table size, Index info      |
| Query Performance    | Top Queries by Time, Frequency      |

**Retention**: Default 7-day metrics retention

**Refresh Rate**: 30 seconds (configurable)

---

### 🔢 Key Commands (For Manual Monitoring)

| Purpose                  | Command or Tool                                     |
| ------------------------ | --------------------------------------------------- |
| Check PostgreSQL status  | `systemctl status postgresql`                       |
| Check replication status | `SELECT * FROM pg_stat_replication;`                |
| View locks and blocks    | `SELECT * FROM pg_locks;`                           |
| Table bloat estimate     | Check pgstattuple or `pg_bloat_check.sql`           |
| Active queries           | `SELECT * FROM pg_stat_activity;`                   |
| Database size            | `SELECT pg_size_pretty(pg_database_size('dbname'))` |

---

### ⚖️ Alert Manager & Silence

* **Alert Conditions**:

  * CPU > 90% = Critical
  * Disk Full = Critical
  * Replication Lag > Threshold
  * Node Exporter Down = Alert
* **Action**:

  * Alert sent to L1 team (via ServiceNow)
  * Escalated to DBAs if unresolved
* **Silence (Blackout)**:

  * Used during maintenance
  * Prevents false alarms
  * Can be configured for 1 hour to N days

---

### 🔧 Examples of Issues Caught

| Issue Type      | Tool/Section            | Resolution Action                  |
| --------------- | ----------------------- | ---------------------------------- |
| High CPU Usage  | Grafana CPU Panel       | Investigate queries, check load    |
| Table Bloat     | Bloat Panel             | Full VACUUM (by DBA)               |
| Query Slowness  | Top Queries by Duration | Optimize query/indexing            |
| Replication Lag | Replication Metrics     | Restart streaming, check WAL files |

---

### ❓ Interview Questions & Answers

**Q1:** What are the tools used for PostgreSQL monitoring?
**A1:** Grafana, Prometheus, PGAdmin, and manual CLI scripts.

**Q2:** What is the purpose of Node Exporter?
**A2:** It collects OS and DB metrics from PostgreSQL hosts for Prometheus.

**Q3:** How do you detect table bloat?
**A3:** Use Grafana bloat panel or queries like pgstattuple to check fragmentation.

**Q4:** What is a 'Silence' in Alertmanager?
**A4:** It is a mechanism to mute alerts during maintenance or planned downtimes.

**Q5:** What is replication lag and where is it shown?
**A5:** The delay between primary and standby DB. Shown in Grafana under Replication Metrics.

---

### 🌐 Reference & Notes

* Tools: Grafana, Prometheus, AlertManager
* PostgreSQL Version: Custom port used (e.g., 9432)
* Only production servers monitored with this setup
* Alerts handled initially by L1 team; escalated as needed

---

Would you like this converted into:

* PDF
* Excel Sheet
* PNG Visual Dashboard
* Grafana Dashboard JSON Export

Let me know if you want PGAdmin examples, PromQL queries, or sample Grafana panels next.


🔭 Prometheus & AlertManager in KOB Architecture
🧱 1. Prometheus in KOB
Prometheus is the core metrics collection and monitoring engine used to monitor PostgreSQL clusters and infrastructure in the KOB platform.

🔹 Prometheus Components in KOB
Component	Role
prometheus-server	Scrapes metrics from exporters and stores time-series data
node-exporter	Collects OS-level metrics (CPU, memory, disk)
postgres-exporter	Exposes PostgreSQL metrics (connections, WALs, replication, locks)
thanos-sidecar	Extends Prometheus by pushing data to object storage (S3)
kube-state-metrics	Collects Kubernetes resource metrics (Pods, Deployments, PVC usage)

⚙️ Prometheus Workflow in KOB
mermaid
Copy
Edit
graph TD
  A[PostgreSQL DB Nodes] -->|Metrics| B[prometheus-postgres-exporter]
  B --> C[Prometheus Server]
  C --> D[Time-Series DB]
  C --> E[AlertManager]
  C --> F[Grafana Dashboard]
  C --> G[Thanos Sidecar]
  G --> H[S3 Object Storage]
  C --> I[Kube-State-Metrics]
🚨 2. AlertManager in KOB
AlertManager handles alerting rules, grouping, notification routing, and silencing.

🔹 AlertManager Components
Component	Description
alertmanager	Receives alerts from Prometheus and sends notifications
config.yaml	Defines routes, receivers, inhibition rules, and templates
silence rules	Temporarily mute alerts (used during maintenance or blackout)
receivers	Email, Slack, ServiceNow, PagerDuty, Webhook

📣 AlertManager Flow in KOB
Prometheus evaluates alert rules (e.g., CPU > 90%, replication lag)

If a condition is breached, Prometheus pushes alert to AlertManager

AlertManager routes it to a receiver (e.g., ServiceNow, Slack)

If in Silence, the alert is muted and not forwarded

L1 teams act, escalate to DBAs if unresolved

🧾 Sample Alert Rules
yaml
Copy
Edit
groups:
- name: postgres.rules
  rules:
  - alert: PostgreSQLHighCPU
    expr: avg by (instance) (rate(process_cpu_seconds_total{job="postgres"}[5m])) > 0.9
    for: 2m
    labels:
      severity: critical
    annotations:
      summary: "High CPU usage on {{ $labels.instance }}"
      description: "CPU usage is above 90% for more than 2 minutes."
🔐 Silence (Blackout) Use in KOB
Use Case	Example
DB Patch Upgrade	Silence created for affected DB node
Storage Maintenance	Mute alerts on PVC/Disk
Weekly Restart	Suppress false “InstanceDown” alerts

🧰 KOB Prometheus Helm Chart Includes:
prometheus-server.yaml

alertmanager.yaml

rules.yaml

thanos-sidecar.yaml

grafana-dashboards/

serviceMonitor and PodMonitor for Kubernetes scraping

📦 Storage and Scaling with Thanos (Optional in KOB)
Component	Role
Thanos Sidecar	Pushes metrics to S3 / long-term object store
Thanos Store	Retrieves historical data from S3
Thanos Querier	Merges data from multiple Prometheus + Thanos stores
Thanos Compactor	Deduplicates and compacts old data

🚨 AlertManager in KOB: Architecture + Silence Logic
🧠 Purpose
AlertManager is used to route and manage alerts triggered by Prometheus based on defined thresholds. It helps ensure issues are escalated properly without spamming teams during maintenance windows.

🔁 KOB Workflow for Alert Handling
mermaid
Copy
Edit
graph TD
  A[Prometheus Rules] -->|Alert Fired| B[AlertManager]
  B --> C{Check Silence?}
  C -- Yes --> D[Suppress Alert (No Action)]
  C -- No --> E[Route Alert]
  E --> F[L1 Team (ServiceNow)]
  F --> G{Resolved?}
  G -- Yes --> H[Incident Closed]
  G -- No --> I[Escalate to DBAs]
🧾 Key Concepts from Session
Concept	Explanation
Silence	Temporary mute (blackout) applied to a DB/server instance during maintenance
Retention	Silences can be configured for hours, days, or even recurring
Reuse	Expired silences can be recreated quickly – no need to start from scratch
Use Case	Applied before shutdown, migration, patching to avoid false alerts
L1 Routing	If not silenced, AlertManager routes alerts (CPU, DB down, WAL lag) to L1
Escalation	If unresolved, L1 escalates incidents to DBAs or Engineering

🛡️ Silence/Blackout Strategy in KOB
Scenario	Silence Needed?	Duration	Reason
DB Patch Upgrade	✅ Yes	2–3 hrs	Avoid false CPU/Disk alerts during restarts
Replication Maintenance	✅ Yes	1–2 hrs	Prevent lag-related alerts
Table Vacuum/Reindex	✅ Yes	30 min–1h	Disk/CPU usage may spike
PostgreSQL Restart	✅ Yes	30 mins	Avoid false “DB Down” or “Exporter Down” alerts

🔧 Silence Example (as discussed)
You can create silence by selecting:

Instance name: mydb01:9100

Duration: 1 hour, 2 hours, 7 days, etc.

Comment: e.g., "Planned upgrade by DBA team"

Silence can be edited, expired, or recreated after expiry

This prevents unnecessary L1 incident creation (especially across systems like MongoDB, SQL, Oracle, Redis)

